[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Humboldt-WI/bads/blob/master/tutorials/9_nb_feature_engineering.ipynb) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/code
!pip install --upgrade pip
!pip install -r requirements.txt


### Parameters and Settings

In [ ]:
# Import standard Python libraries


import numpy as np
import pandas as pd
import time
import json
import os
import random
import joblib

from sklearn.model_selection import train_test_split
#from sklearn.datasets import fetch_covtype
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import pairwise_kernels
from tqdm.notebook import tqdm
from os import path
import warnings
from pickle import load
warnings.filterwarnings("ignore")

import keras
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, LSTM, Concatenate
from tensorflow.keras.layers import Dropout, Bidirectional, Flatten
from tensorflow.keras.layers import Conv1D, Conv2D
from tensorflow.keras.layers import MaxPooling1D, Embedding
from tensorflow.keras.layers import RepeatVector, Reshape
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
#from sklearn.neural_network import MLPRegressor
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import xgboost as xgb


In [ ]:
%cd /content/drive/MyDrive/code/vest-python-master/

/content/drive/MyDrive/code/vest-python-master


In [ ]:
from gtda.time_series import TakensEmbedding

from vest.models.univariate import UnivariateVEST
from vest.config.aggregation_functions \
    import (SUMMARY_OPERATIONS_ALL,
            SUMMARY_OPERATIONS_FAST,
            SUMMARY_OPERATIONS_SMALL)
# In case of an error-message like following:
# ContextualVersionConflict: (scipy 1.4.1 (/usr/local/lib/python3.7/dist-packages), Requirement.parse('scipy>=1.5'), {'stumpy'})
# replace the float_factorial function in the scipy package (most likely in this location: /usr/local/lib/python3.7/dist-packages/scipy/_lib/_util.py )
# with the following function:

#def float_factorial(num):
#    """Compute the factorial and return as a float
#
#    Returns infinity when result is too large for a double
#    """
#    val = float(math.factorial(num)) if num < 171 else np.inf
#    return val

# Afterwards restart runtime and run code again.

#### Helper Functions

In [ ]:
def get_accuracy(y_real, y_pred):
  deviation_pred = np.abs((y_real - y_pred))
  dev_in_percent = (deviation_pred/100)*np.abs(y_real)
  dev_in_percent = np.where(dev_in_percent==0, 5, dev_in_percent)
  acc_5_lst = np.where(dev_in_percent>5, 0, 1)
  acc_5 = acc_5_lst.sum()/len(y_real)
  return acc_5

def get_accuracy_90(y_real, y_pred):
  deviation_pred = np.abs((y_real - y_pred))
  dev_in_percent = (deviation_pred/100)*np.abs(y_real)
  dev_in_percent = np.where(dev_in_percent==0, 5, dev_in_percent)
  acc_5_lst = np.where(dev_in_percent>5, 0, 1)
  acc_10 = acc_5_lst.sum()/len(y_real)
  return acc_10

def get_wmape(y_real, y_pred):
  y_real, y_pred = np.array(y_real), np.array(y_pred) 
  return np.sum(np.abs((y_real - y_pred))) / np.sum(np.abs(y_real))


def get_mase_onestep(y_real, y_pred, y_train):
    """
    Computes the MEAN-ABSOLUTE SCALED ERROR forcast error for univariate time series prediction.
    
    See "Another look at measures of forecast accuracy", Rob J Hyndman
    
    parameters:
        training_series: the series used to train the model, 1d numpy array
        testing_series: the test series to predict, 1d numpy array or float
        prediction_series: the prediction of testing_series, 1d numpy array (same size as testing_series) or float
        absolute: "squares" to use sum of squares and root the result, "absolute" to use absolute values.
    
    """
    n = y_train.shape[0]
    d = np.abs(np.diff(y_train)).sum()/(n-1)
    
    errors = np.abs(y_real - y_pred )
    return errors.mean()/d

def get_mase(actual: np.ndarray, predicted: np.ndarray, naive: np.ndarray):
    """
    Mean Absolute Scaled Error
    Baseline (benchmark) is computed with naive forecasting 24 h
    """
    return mean_absolute_error(actual, predicted) / mean_absolute_error(actual, naive)


In [ ]:
# Formulas to get all evaluation metrics for flattened and unflattened predictions
def evaluate_predictions_flat(actual, predicted, test_y=None, testing=True):

    if (actual.shape[1] != 1) or (predicted.shape[1] != 1):
      raise AssertionError('Second dimension of actual and predicted array must be 1')

    rmse_flat = mean_squared_error(actual, predicted, squared=False)
    nrmse_flat = np.asscalar(mean_squared_error(actual, predicted, squared=False)/(max(actual)-min(actual)))
    mae_flat = mean_absolute_error(actual, predicted)
    wmape_flat = get_wmape(actual, predicted)
    mase_flat = get_mase(actual, predicted, test_y)
    acc_flat = get_accuracy(actual, predicted)
    acc_90_flat = get_accuracy_90(actual, predicted)

    print('RMSE/MAE/wMAPE/MASE/NRMSE/ACCURACY')
    print(f'{rmse_flat:.2f},{mae_flat:.2f},{wmape_flat:.2f},{mase_flat:.2f}, {nrmse_flat:.2f}, {acc_flat:.2f}')

    if not round(wmape_flat,0)==0 and testing==True:
      raise AssertionError()

    return [rmse_flat, mae_flat, wmape_flat, mase_flat, nrmse_flat, acc_flat, acc_90_flat]

def evaluate_predictions_windows(actual, predicted, test_y=None, testing=True):

    wmapes_24 = []
    rmse_24 = []
    nrmse_24 = []
    mae_24 = []
    mase_24 = []
    acc_24 = []
    acc_90_24 = []

    for i in range(actual.shape[0]):
      rmses = mean_squared_error(actual[i], predicted[i], squared=False)
      nrmses = mean_squared_error(actual[i], predicted[i], squared=False)/(max(actual[i])-min(actual[i]))
      maes = mean_absolute_error(actual[i], predicted[i])
      wmapes = get_wmape(actual[i], predicted[i])
      mases = get_mase(actual[i], predicted[i], test_y[i])
      accs = get_accuracy(actual[i], predicted[i])
      accs_90 = get_accuracy_90(actual[i], predicted[i])

      rmse_24.append(rmses)
      nrmse_24.append(nrmses)
      mae_24.append(maes)
      wmapes_24.append(wmapes)
      mase_24.append(mases)
      acc_24.append(accs)
      acc_90_24.append(accs_90)

    if not round(np.mean(np.ma.masked_invalid(wmapes_24)),0)==0 and testing==True:
      raise AssertionError()

    return [rmse_24, mae_24, wmapes_24, mase_24, nrmse_24, acc_24, acc_90_24]

In [ ]:
# calculate differences to original data for many models:
def print_originalvalues_results(result_list, y_test_control, naive, originals=True):
  
  rmse_lst = []
  mae_lst = []
  wmape_lst = []
  mase_lst = []
  nrmse_lst = []
  accur_lst = []
  accur_90_lst = []

  for i in range(len(result_list)):
    if originals == True:
      y_pred_fridge = preproc_target.inverse_transform(result_list[i].reshape((-1,1)))
      metric_list = evaluate_predictions_flat(y_test_control, y_pred_fridge, naive, testing=False)
    else:
      scaled_true = preproc_target.transform(y_test_control)
      scaled_naive = preproc_target.transform(naive)
      metric_list = evaluate_predictions_flat(scaled_true, result_list[i].reshape((-1,1)), scaled_naive, testing=False)

    
    rmse_lst.append(metric_list[0])
    mae_lst.append(metric_list[1])
    wmape_lst.append(metric_list[2])
    mase_lst.append(metric_list[3])
    nrmse_lst.append(metric_list[4])
    accur_lst.append(metric_list[5])
    accur_90_lst.append(metric_list[6])

  return rmse_lst, mae_lst, wmape_lst, mase_lst, nrmse_lst, accur_lst, accur_90_lst

In [ ]:
# Fil empty dataframe with the time lag observations per variable as features for 24h
def get_tlags_rolling_features(df, col_names, tlag):

   dataframe = pd.DataFrame()
   for col in col_names:
     for i in range((tlag-1),0,-1):
       dataframe[col + '_t-'+str(i)] = df[col].shift(i).values[:]
     dataframe[col] = df[col]
   
   dataframe = dataframe[(tlag-1):]
   dataframe.dropna(inplace=True)
   dataframe.reset_index(drop=True, inplace=True)
   
   nparray = np.empty((dataframe.shape[0], tlag, len(col_names)))
   for i in range(len(col_names)):
      nparray[:,:,i] = dataframe.values[:,i*tlag:(i+1)*tlag]
   return dataframe, nparray

In [ ]:
def get_msvr_gamma(xtrain, relational=True, factor=10):

  if relational==True:
    msvr_gamma = 1 / (xtrain.shape[1] * xtrain.var())

  else:
    msvr_gamma = 1 / factor
  
  return msvr_gamma

#### Specify Experiment Parameters

In [ ]:
# Load Data
dataset = 'refit' # 'refit' 'ampds' , 'pecansd', 'greend'
dimension_var = 'many' # define wether it will be the baseline or a feature run-through
ylag = 24 # define number of time steps to predict
target = 'Fridge' # Specify the target Appliance
feature_group = 'baseline' 
# Options. 'baseline', 'appliances', 'weather', 'sinecosine', 'lon_loff', 'all', 
# 'weather_sinecosine', 'autoreg_features', 'interact_features', 'vest_features', 'taken_features'
path_to_data_folder = '/content/drive/MyDrive/'

if dataset == 'refit':
  data_url = f'{path_to_data_folder}data/REFIT/'
  reshape_dimension = 113
elif dataset == 'ampds':
  data_url = f'{path_to_data_folder}data/AMPds/'
  reshape_dimension = 113
elif dataset == 'pecansd':
  data_url = f'{path_to_data_folder}data/PecanSD/'
  reshape_dimension = 86
elif dataset == 'greend':
  data_url = f'{path_to_data_folder}data/GreenD/'
  reshape_dimension = 86  
else:
  raise AssertionError('Please specify a valid dataset')

df = pd.read_csv(f'{data_url}traindata_scaled.csv')

nparray = np.loadtxt(f'{data_url}traindata_scaled.txt')
nparray = nparray.reshape((-1, ylag, reshape_dimension))
print(nparray.shape)

# train and valid set params
split_ratio = 0.2
val_split = int(np.floor(len(df)*split_ratio))
target_idx = list(df.columns).index(target)-1

# FE Selection Features:
if feature_group == 'baseline':
  col_idx = np.r_[target_idx:target_idx+1]
  dimension_var = 'one'
elif feature_group == 'appliances':
  if dataset in ('refit', 'ampds'):
    # Nur Appliances:
    col_idx = np.r_[0:5]
  elif dataset in ('pecansd', 'greend'):
    col_idx = np.r_[0:4]
 
elif feature_group == 'weather': 
  if dataset in ('refit', 'ampds'):
    col_idx = np.r_[target_idx:target_idx+1,5:8]
  elif dataset in ('pecansd', 'greend'):
    col_idx = np.r_[target_idx:target_idx+1,4:7]

elif feature_group == 'sinecosine': 
  if dataset in ('refit', 'ampds'):
    col_idx = np.r_[target_idx:target_idx+1,90:91, 95:103]
  elif dataset in ('pecansd', 'greend'):
    col_idx = np.r_[target_idx:target_idx+1, 65:66, 70:78]

elif feature_group == 'lon_loff': 
  if dataset in ('refit', 'ampds'):
    col_idx = np.r_[target_idx:target_idx+1, 103+target_idx:105+target_idx+1]
  elif dataset in ('pecansd', 'greend'):
    col_idx = np.r_[target_idx:target_idx+1, 78+target_idx:80+target_idx+1]

elif feature_group == 'all': 
  if dataset in ('refit', 'ampds'):
    col_idx = np.r_[0:91, 95:113]
  elif dataset in ('pecansd', 'greend'):
    col_idx = np.r_[0:66, 70:86] 

elif feature_group == 'weather_sinecosine': 
  if dataset in ('refit', 'ampds'):
    col_idx = np.r_[target_idx:target_idx+1, 90:91, 95:103]
  elif dataset in ('pecansd', 'greend'):
    col_idx = np.r_[target_idx:target_idx+1,65:66, 70:78] 
elif feature_group =='autoreg_features': 
  autoreg_features = True
elif feature_group =='interact_features':
  interact_features = True
elif feature_group =='vest_features':
  vest_features = True
elif feature_group =='taken_features':
  taken_features = True  
else:
  raise AssertionError('Please specify a valid feature group')


# Model Paramteres
epochs = 10
loss = 'mse'
optimizer = tf.keras.optimizers.Adam(lr=0.0001)

drpt_rate = 0.06
hidd_dim = 172
lr_alpha = 0.2

# XGBoost
num_round = 750

# Leave most parameters as default
param = {'objective': 'reg:squarederror', # Specify multiclass classification
         'tree_method': 'gpu_hist' # Use GPU accelerated algorithm
         }
#param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}

# MSVR
msvr_kernel = 'rbf'
msvr_relational = True # describe wofür param ist:
msvr_factor = 10 # describe wofür das ist
msvr_epsilon=0.1 
msvr_degree=2
msvr_c=0.1

# LSTM

#Seq2Seq context
hidd_dim_context = 281

# FFNN:
hidd_dim_ffnn = 182
batch_size = 64

#CNN-LSTM
hidd_dim_cnn_lstm = 146
conv_filter = 64
drpt_rate_cnn = 0.36 # check ob das im paper steht sonst anpassen
pl_size = 2
kernel_size = 2

# Evaluation Dictionaryy:
evaluation_dict = {}


(9330, 24, 113)


### Data Selection and Feature Groups

In [ ]:
Xtrain_arrayfe = nparray[:-val_split]
ytrain_arrayfe = nparray[ylag:-val_split+ylag, :, target_idx]

Xvalid_arrayfe = nparray[-val_split:-ylag]
yvalid_arrayfe = nparray[-val_split+ylag:, :, target_idx]

# get scalars and test transformation process for evaluation
preproc_target = load(open(f'{data_url}std_scaler_{target}.pkl', 'rb'))
df_tr = pd.read_csv(f'{data_url}original_traindata_imputed_outlreplaced.csv')

# inverse transform our predictions:
y_valid_fridge = yvalid_arrayfe[::24, :].reshape((-1,1))
y_valid_fridge = preproc_target.inverse_transform(y_valid_fridge)

#inverse transform test values
Xvalid_fridge = Xvalid_arrayfe[::24,:,target_idx].reshape((-1,1))
Xvalid_fridge = preproc_target.inverse_transform(Xvalid_fridge)

# get part of original data that we use
df_valid_true = df_tr[target].iloc[-len(y_valid_fridge)-11:-11].values.reshape((-1,1)) # refit = -3/ smart -13/ ampds -13

# calculate difference to original data
rmse, mae, mape_sin_nan, mase, nrmse, acc, acc_90 = evaluate_predictions_flat(df_valid_true, y_valid_fridge, test_y= Xvalid_fridge)

RMSE/MAE/wMAPE/MASE/NRMSE/ACCURACY
0.00,0.00,0.00,0.00, 0.00, 1.00


In [ ]:
# Load Test Data
test_file = f'{data_url}testdata_scaled.txt'
if path.exists(test_file):
  Xtest_array = np.loadtxt(test_file)
else:
  raise AssertionError(f"File does not exist run notebook data_preparation before with dataset = {dataset} and train_test = 'test")

Xtest_array = Xtest_array.reshape((-1,24,reshape_dimension)) 
Xtest = Xtest_array[:-1, :, :].copy()
ytest = Xtest_array[1:, :, target_idx].copy()

# Check for Testdata:
df_t = pd.read_csv(f'{data_url}original_testdata_imputed_outlreplaced.csv')

y_test_fridge = ytest.reshape((-1,1))
y_test_fridge = preproc_target.inverse_transform(y_test_fridge)
y_test_fridge_wind = y_test_fridge.reshape(ytest.shape)

#inverse transform test values
Xtest_fridge = Xtest[:,:,target_idx].reshape((-1,1))
Xtest_fridge = preproc_target.inverse_transform(Xtest_fridge)
Xtest_fridge_wind = Xtest_fridge.reshape(Xtest[:,:,0].shape)

# get part of original data that we use
df_test_true = df_t[target].iloc[-len(y_test_fridge)-24:-24].values.reshape((-1,1)) # refit -24  / Ampds -24
df_test_true_wind = df_test_true.reshape(Xtest[:,:,0].shape)

# calculate difference to original data
rmse, mae, mape_sin_nan, mase, nrmse, acc, acc_90 = evaluate_predictions_flat(df_test_true, y_test_fridge, test_y = Xtest_fridge)

RMSE/MAE/wMAPE/MASE/NRMSE/ACCURACY
0.00,0.00,0.00,0.00, 0.00, 1.00


#### Feature Group Selection

##### Onehot encoding

In [ ]:
#Nur Fridge
#nparray_train = np.loadtxt(f'{data_url}traindata/traindata_onehot.txt')
#nparray_train = nparray_train.reshape((-1, ylag, 48))
#Xtrain_arrayfe = nparray_train[:-val_split]
#
#Xvalid_arrayfe = nparray_train[-val_split:-ylag]
#
## Load Test Data
#nparray_test = np.loadtxt(f'{data_url}testdata/testdata_onehot.txt')
#nparray_test = nparray_test.reshape((-1,24,48)) 
#Xtest = nparray_test[1:-1, :, :].copy()
#
#X_train = Xtrain_arrayfe.reshape((Xtrain_arrayfe.shape[0],-1))
#X_test = Xtest.reshape((Xtest.shape[0],-1))
#
#print(f'Shape of train and val array: {Xtrain_arrayfe.shape}, {Xvalid_arrayfe.shape}')
#print(f'Shape of train and val y array: {ytrain_arrayfe.shape}, {yvalid_arrayfe.shape}')
#print(f'Shape of test data X-array and y-array: {Xtest.shape}, {ytest.shape}') 

##### Keep Feature Group Selection

In [ ]:
# Target timeseries + past stat summary features 
if 'autoreg_features' in locals():
  candidate_list = [s for s in df.columns.to_list() if target in s]
  fe_list = candidate_list[:9]
# Fridge timeseries + past summary features with other appliances (interaction variables)
elif 'interact_features' in locals():
    candidate_list = [s for s in df.columns.to_list() if target in s]
    fe_list = candidate_list[9:-2]
    fe_list.insert(0,candidate_list[target_idx])
else:
  pass

if 'fe_list' in locals():
  col_idx = []
  for col in fe_list:
    pos = df.columns.tolist().index(col)-1
    col_idx.append(pos)
  col_idx.sort()

In [ ]:
Xtrain_arrayfe = Xtrain_arrayfe[:,:,col_idx]
Xvalid_arrayfe = Xvalid_arrayfe[:,:,col_idx]
Xtest = Xtest[:,:,col_idx]

# Input XGBoost, MSVR and FFNN:
X_train = Xtrain_arrayfe.reshape((Xtrain_arrayfe.shape[0],-1))
X_valid = Xvalid_arrayfe.reshape((Xvalid_arrayfe.shape[0],-1))
X_test = Xtest.reshape((Xtest.shape[0],-1))

print(f'Shape of train and val array: {Xtrain_arrayfe.shape}, {Xvalid_arrayfe.shape}')
print(f'Shape of train and val y array: {ytrain_arrayfe.shape}, {yvalid_arrayfe.shape}')
print(f'Shape of test data X-array and y-array: {Xtest.shape}, {ytest.shape}') 
print(X_train.shape)
print(X_test.shape)

Shape of train and val array: (7446, 24, 1), (1860, 24, 1)
Shape of train and val y array: (7446, 24), (1860, 24)
Shape of test data X-array and y-array: (112, 24, 1), (112, 24)
(7446, 24)
(112, 24)


In [ ]:
# Feature shapes for Seq2Seq context:
if dimension_var == 'one':
  # encoder inputs:
  Xtrain_arrayfe_enc = Xtrain_arrayfe[:,:,:]
  Xvalid_arrayfe_enc = Xvalid_arrayfe[:,:,:]
  Xtest_enc = Xtest[:,:,:]
else:
  # encoder inputs:
  Xtrain_arrayfe_enc = Xtrain_arrayfe[:,:,1:]
  Xvalid_arrayfe_enc = Xvalid_arrayfe[:,:,1:]
  Xtest_enc = Xtest[:,:,1:]

# decoder output:
Xtrain_arrayfe_dec = Xtrain_arrayfe[:,:,0:1]
Xvalid_arrayfe_dec = Xvalid_arrayfe[:,:,0:1]
Xtest_dec = Xtest[:,:,0:1]


# Feature Shapes for CNN Input Layer:
Xtrain_arrayfe_cnn_lstm = Xtrain_arrayfe.reshape((Xtrain_arrayfe.shape[0],Xtrain_arrayfe.shape[1],Xtrain_arrayfe.shape[2],1))
Xvalid_arrayfe_cnn_lstm = Xvalid_arrayfe.reshape((Xvalid_arrayfe.shape[0],Xvalid_arrayfe.shape[1],Xvalid_arrayfe.shape[2],1))
Xtest_cnn_lstm = Xtest.reshape((Xtest.shape[0],Xtest.shape[1],Xtest.shape[2],1))

##### VEST Autoregressive features and ts transformations

In [ ]:
if 'vest_features' in locals():
  vest_model = UnivariateVEST()

  vest_model.fit(X=Xtrain_arrayfe[:,:,0],
            correlation_thr=0.8,
            apply_transform_operators=True,
            summary_operators=SUMMARY_OPERATIONS_SMALL)

  vestfe_train = vest_model.dynamics
  vestfe_train_ar = vestfe_train.values
  X_tr_augmented = np.concatenate((Xtrain_arrayfe[:,:,0], vestfe_train_ar), axis=1)

  vestfe_valid = vest_model.transform(Xvalid_arrayfe[:,:,0])
  vestfe_valid_ar = vestfe_valid.values
  X_vl_augmented = np.concatenate((Xvalid_arrayfe[:,:,0], vestfe_valid_ar), axis=1)

  vestfe_test = vest_model.transform(Xtest[:,:,0])
  vestfe_test_ar = vestfe_test.values
  X_ts_augmented = np.concatenate((Xtest[:,:,0], vestfe_test_ar), axis=1)

  Xtrain_arrayfe = X_tr_augmented.reshape((X_tr_augmented.shape[0], X_tr_augmented.shape[1], 1))
  Xvalid_arrayfe = X_vl_augmented.reshape((X_vl_augmented.shape[0], X_vl_augmented.shape[1], 1))
  Xtest = X_ts_augmented.reshape((X_ts_augmented.shape[0], X_ts_augmented.shape[1], 1))

  # Input RF and MSVR:
  X_train = Xtrain_arrayfe.reshape((Xtrain_arrayfe.shape[0],-1))
  X_valid = Xvalid_arrayfe.reshape((Xvalid_arrayfe.shape[0],-1))
  X_test = Xtest.reshape((Xtest.shape[0],-1))

  # Vest feature shapes for Seq2Seq context
  Xtrain_arrayfe_enc = Xtrain_arrayfe[:,0:24,:]
  Xvalid_arrayfe_enc = Xvalid_arrayfe[:,0:24,:]
  Xtest_enc = Xtest[:,0:24,:]
  
  Xtrain_arrayfe_dec = Xtrain_arrayfe[:,24:,:]
  Xvalid_arrayfe_dec = Xvalid_arrayfe[:,24:,:]
  Xtest_dec = Xtest[:,24:,:]

else:
  pass

In [ ]:
print(f'Shape of train and val array: {Xtrain_arrayfe.shape}, {Xvalid_arrayfe.shape}')
print(f'Shape of train and val y array: {ytrain_arrayfe.shape}, {yvalid_arrayfe.shape}')
print(f'Shape of test data X-array and y-array: {Xtest.shape}, {ytest.shape}') 
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)
# assertion test einfügen dass error gibt wenn falsch

Shape of train and val array: (7446, 24, 1), (1860, 24, 1)
Shape of train and val y array: (7446, 24), (1860, 24)
Shape of test data X-array and y-array: (112, 24, 1), (112, 24)
(7446, 24)
(1860, 24)
(112, 24)


##### Taken Transformation

In [ ]:
if 'taken_features' in locals():

  TE = TakensEmbedding(time_delay=1, dimension=3, flatten=True)

  # One dimensional case
  if dimension_var == 'one':
    Xtrain_arrayfe = TE.fit_transform(Xtrain_arrayfe.reshape((Xtrain_arrayfe.shape[0], Xtrain_arrayfe.shape[1])))
    Xvalid_arrayfe = TE.transform(Xvalid_arrayfe.reshape((Xvalid_arrayfe.shape[0], Xvalid_arrayfe.shape[1])))
    Xtest = TE.transform(Xtest.reshape((Xtest.shape[0], Xtest.shape[1])))
    print(f'Shapes of train, valid and test input: {Xtrain_arrayfe.shape}, {Xvalid_arrayfe.shape}, {Xtest.shape}')
  else:
    Xtrain_arrayfe = TE.fit_transform(np.transpose(Xtrain_arrayfe, (0, 2, 1)))
    Xvalid_arrayfe = TE.transform(np.transpose(Xvalid_arrayfe, (0, 2, 1)))
    Xtest = TE.transform(np.transpose(Xtest, (0, 2, 1)))
    print(f'Shapes of train, valid and test input: {Xtrain_arrayfe.shape}, {Xvalid_arrayfe.shape}, {Xtest.shape}')  
  # For RF and MSVR
  X_train = Xtrain_arrayfe.reshape((Xtrain_arrayfe.shape[0], -1))
  X_test = Xtest.reshape((Xtest.shape[0], -1))

print(f'Shape of train and val array: {Xtrain_arrayfe.shape}, {Xvalid_arrayfe.shape}')
print(f'Shape of train and val y array: {ytrain_arrayfe.shape}, {yvalid_arrayfe.shape}')
print(f'Shape of test data X-array and y-array: {Xtest.shape}, {ytest.shape}') 
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

Shape of train and val array: (7446, 24, 1), (1860, 24, 1)
Shape of train and val y array: (7446, 24), (1860, 24)
Shape of test data X-array and y-array: (112, 24, 1), (112, 24)
(7446, 24)
(1860, 24)
(112, 24)


### Model Training


#### XGBOOST

In [ ]:
tmp = time.time()
results_lst = []
for i in range(24): #ytrain_arrayfe.shape[1]
  # Convert input data from numpy to XGBoost format
  print(f'Starting Model: {i}')
  dtrain = xgb.DMatrix(X_train, label=ytrain_arrayfe[:,i])
  dvalid = xgb.DMatrix(X_valid, label=yvalid_arrayfe[:,i])
  dtest = xgb.DMatrix(X_test, label=ytest[:,i])

  # specify validations set to watch performance
  watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
  
  # Train model
  xgbtrained = xgb.train(param, dtrain, num_round, watchlist, verbose_eval=False) #

  #save model
  joblib.dump(xgbtrained, f'{data_url}xgb_model.dat')
  xgbtrained.__del__()

  #load saved model
  xgbtrained = joblib.load(f'{data_url}xgb_model.dat')

  predictions = xgbtrained.predict(dtest)
  results_lst.append(predictions.reshape((predictions.shape[0], 1)))
  
xgb_train_time = time.time() - tmp
print("GPU Training Time: %s seconds" % (str(xgb_train_time)))


Starting Model: 0
Starting Model: 1
Starting Model: 2
Starting Model: 3
Starting Model: 4
Starting Model: 5
Starting Model: 6
Starting Model: 7
Starting Model: 8
Starting Model: 9
Starting Model: 10
Starting Model: 11
Starting Model: 12
Starting Model: 13
Starting Model: 14
Starting Model: 15
Starting Model: 16
Starting Model: 17
Starting Model: 18
Starting Model: 19
Starting Model: 20
Starting Model: 21
Starting Model: 22
Starting Model: 23
GPU Training Time: 124.41970920562744 seconds


In [ ]:
y_pred = np.hstack(results_lst)
y_pred_fridge = y_pred.reshape((-1,1))
y_pred_fridge = preproc_target.inverse_transform(y_pred_fridge)

y_pred_wind = y_pred_fridge.reshape(df_test_true_wind.shape)

evaluation_dict['XGBoost'] = evaluate_predictions_flat(df_test_true, y_pred_fridge, test_y = Xtest_fridge, testing=False)
evaluation_dict['XGBoost'].append(xgb_train_time)


RMSE/MAE/wMAPE/MASE/NRMSE/ACCURACY
17.64,14.45,0.91,0.86, 0.17, 0.80


#### MSVR

##### Code

In [ ]:
class MSVR():
    def __init__(self, kernel='rbf', degree=3, gamma=None, coef0=0.0, tol=0.001, C=1.0, epsilon=0.1):
        super(MSVR, self).__init__()
        self.kernel = kernel
        self.degree = degree
        self.gamma = gamma
        self.coef0 = coef0
        self.tol = tol
        self.C = C
        self.epsilon = epsilon
        self.Beta = None
        self.NSV = None
        self.xTrain = None

    def fit(self, x, y):
        self.xTrain = x.copy()
        C = self.C
        epsi = self.epsilon
        tol = self.tol

        n_m = np.shape(x)[0]  # num of samples
        n_d = np.shape(x)[1]  # input data dimensionality
        n_k = np.shape(y)[1]  # output data dimensionality (output variables)

        # H = kernelmatrix(ker, x, x, par)
        H = pairwise_kernels(x, x, metric=self.kernel, filter_params=True,
                             degree=self.degree, gamma=self.gamma, coef0=self.coef0)

        self.Beta = np.zeros((n_m, n_k))

        #E = prediction error per output (n_m * n_k)
        E = y - np.dot(H, self.Beta)
        #RSE
        u = np.sqrt(np.sum(E**2, 1, keepdims=True))

        #RMSE
        RMSE = []
        RMSE_0 = np.sqrt(np.mean(u**2))
        RMSE.append(RMSE_0)

        #points for which prediction error is larger than epsilon
        i1 = np.where(u > epsi)[0]

        #set initial values of alphas a (n_m * 1)
        a = 2 * C * (u - epsi) / u

        #L (n_m * 1)
        L = np.zeros(u.shape)

        # we modify only entries for which  u > epsi. with the sq slack
        L[i1] = u[i1]**2 - 2 * epsi * u[i1] + epsi**2

        #Lp is the quantity to minimize (sq norm of parameters + slacks)
        Lp = []
        BetaH = np.dot(np.dot(self.Beta.T, H), self.Beta)
        Lp_0 = np.sum(np.diag(BetaH), 0) / 2 + C * np.sum(L)/2
        Lp.append(Lp_0)

        eta = 1
        k = 1
        hacer = 1
        val = 1

        while(hacer):
            Beta_a = self.Beta.copy()
            E_a = E.copy()
            u_a = u.copy()
            i1_a = i1.copy()

            M1 = H[i1][:, i1] + \
                np.diagflat(1/a[i1]) + 1e-10 * np.eye(len(a[i1]))

            #compute betas
            #       sal1 = np.dot(np.linalg.pinv(M1),y[i1])  #求逆or广义逆（M-P逆）无法保证M1一定是可逆的？
            sal1 = np.dot(np.linalg.inv(M1), y[i1])

            eta = 1
            self.Beta = np.zeros(self.Beta.shape)
            self.Beta[i1] = sal1.copy()

            #error
            E = y - np.dot(H, self.Beta)
            #RSE
            u = np.sqrt(np.sum(E**2, 1)).reshape(n_m, 1)
            i1 = np.where(u >= epsi)[0]

            L = np.zeros(u.shape)
            L[i1] = u[i1]**2 - 2 * epsi * u[i1] + epsi**2

            #%recompute the loss function
            BetaH = np.dot(np.dot(self.Beta.T, H), self.Beta)
            Lp_k = np.sum(np.diag(BetaH), 0) / 2 + C * np.sum(L)/2
            Lp.append(Lp_k)

            #Loop where we keep alphas and modify betas
            while(Lp[k] > Lp[k-1]):
                eta = eta/10
                i1 = i1_a.copy()

                self.Beta = np.zeros(self.Beta.shape)
                #%the new betas are a combination of the current (sal1)
                #and of the previous iteration (Beta_a)
                self.Beta[i1] = eta*sal1 + (1-eta)*Beta_a[i1]

                E = y - np.dot(H, self.Beta)
                u = np.sqrt(np.sum(E**2, 1)).reshape(n_m, 1)

                i1 = np.where(u >= epsi)[0]

                L = np.zeros(u.shape)
                L[i1] = u[i1]**2 - 2 * epsi * u[i1] + epsi**2
                BetaH = np.dot(np.dot(self.Beta.T, H), self.Beta)
                Lp_k = np.sum(np.diag(BetaH), 0) / 2 + C * np.sum(L)/2
                Lp[k] = Lp_k

                #stopping criterion 1
                if(eta < 1e-16):
                    Lp[k] = Lp[k-1] - 1e-15
                    self.Beta = Beta_a.copy()

                    u = u_a.copy()
                    i1 = i1_a.copy()

                    hacer = 0

            #here we modify the alphas and keep betas
            a_a = a.copy()
            a = 2 * C * (u - epsi) / u

            RMSE_k = np.sqrt(np.mean(u**2))
            RMSE.append(RMSE_k)

            if((Lp[k-1]-Lp[k])/Lp[k-1] < tol):
                hacer = 0

            k = k + 1

            #stopping criterion #algorithm does not converge. (val = -1)
            if(len(i1) == 0):
                hacer = 0
                self.Beta = np.zeros(self.Beta.shape)
                val = -1

        self.NSV = len(i1)

    def predict(self, x):
        H = pairwise_kernels(x, self.xTrain, metric=self.kernel, filter_params=True,
                             degree=self.degree, gamma=self.gamma, coef0=self.coef0)
        yPred = np.dot(H, self.Beta)
        return yPred

    # def score(self,x):

##### Predictions

In [ ]:
# define model
start = time.time()
# Train Model 10 Fold:
msvr_result_lst = []
for i in [42, 10, 567, 239, 400, 1390, 380, 9, 27, 769]:
  
  # Training loop
  seed_value = i
  os.environ['PYTHONHASHSEED']=str(seed_value)
  random.seed(seed_value)
  np.random.seed(seed_value)
  msvr = MSVR(kernel = msvr_kernel, gamma = get_msvr_gamma(X_train, relational=msvr_relational, factor=msvr_factor), epsilon= msvr_epsilon, degree= msvr_degree, C= msvr_c) 
  msvr.fit(X_train, ytrain_arrayfe)
  y_pred = msvr.predict(X_test)
  msvr_result_lst.append(y_pred) 
end = time.time()

msvr_train_time = end - start

In [ ]:
rmse_lst_msvr, mae_lst_msvr, wmape_lst_msvr, mase_lst_msvr, nrmse_lst_msvr, acc_lst_msvr, acc_90_lst_msvr = print_originalvalues_results(msvr_result_lst, df_test_true, Xtest_fridge)

msvr_rmse = sum(rmse_lst_msvr)/len(rmse_lst_msvr)
msvr_mae = sum(mae_lst_msvr)/len(mae_lst_msvr)
msvr_wmape = sum(wmape_lst_msvr)/len(wmape_lst_msvr)
msvr_mase = sum(mase_lst_msvr)/len(mase_lst_msvr)
msvr_nrmse = sum(nrmse_lst_msvr)/len(nrmse_lst_msvr)
msvr_acc = sum(acc_lst_msvr)/len(acc_lst_msvr)
msvr_acc_90 = sum(acc_90_lst_msvr)/len(acc_90_lst_msvr)

msvr_lst_results = [msvr_rmse, msvr_mae, msvr_wmape, msvr_mase, msvr_nrmse, msvr_acc, msvr_acc_90, msvr_train_time]
evaluation_dict['MSVR'] = msvr_lst_results

RMSE/MAE/wMAPE/MASE/NRMSE/ACCURACY
15.72,13.52,0.85,0.80, 0.16, 0.78
RMSE/MAE/wMAPE/MASE/NRMSE/ACCURACY
15.72,13.52,0.85,0.80, 0.16, 0.78
RMSE/MAE/wMAPE/MASE/NRMSE/ACCURACY
15.72,13.52,0.85,0.80, 0.16, 0.78
RMSE/MAE/wMAPE/MASE/NRMSE/ACCURACY
15.72,13.52,0.85,0.80, 0.16, 0.78
RMSE/MAE/wMAPE/MASE/NRMSE/ACCURACY
15.72,13.52,0.85,0.80, 0.16, 0.78
RMSE/MAE/wMAPE/MASE/NRMSE/ACCURACY
15.72,13.52,0.85,0.80, 0.16, 0.78
RMSE/MAE/wMAPE/MASE/NRMSE/ACCURACY
15.72,13.52,0.85,0.80, 0.16, 0.78
RMSE/MAE/wMAPE/MASE/NRMSE/ACCURACY
15.72,13.52,0.85,0.80, 0.16, 0.78
RMSE/MAE/wMAPE/MASE/NRMSE/ACCURACY
15.72,13.52,0.85,0.80, 0.16, 0.78
RMSE/MAE/wMAPE/MASE/NRMSE/ACCURACY
15.72,13.52,0.85,0.80, 0.16, 0.78


#### LSTM

In [ ]:
start = time.time()
# Train Model 10 Fold:
lstm_result_lst = []
for i in [42, 10, 567, 239, 400, 1390, 380, 9, 27, 769]:
  # Define input layers
  sequential_input = Input(Xtrain_arrayfe[0].shape)
  
  # Model Architecture: LSTM
  lstm1 = LSTM(hidd_dim, return_sequences=True, input_shape=Xtrain_arrayfe.shape)(sequential_input)
  lr1 = LeakyReLU(alpha=lr_alpha)(lstm1)
  drpt1 = Dropout(drpt_rate)(lr1)
  lstm2 = LSTM(hidd_dim)(drpt1)
  lr2 = LeakyReLU(alpha=lr_alpha)(lstm2)
  drpt2 = Dropout(drpt_rate)(lr2)

  dense = Dense(24)(drpt2)
  outputs = LeakyReLU(alpha=lr_alpha)(dense)
  
  # Define the model
  lstm = Model(inputs=sequential_input, outputs=outputs)
  lstm.compile(loss=loss, optimizer=optimizer)
  #lstm.compile(loss = [loss1,loss2], loss_weights = [l1,l2], optimizer=optimizer)
  
  # Training loop
  seed_value = i
  os.environ['PYTHONHASHSEED']=str(seed_value)
  random.seed(seed_value)
  np.random.seed(seed_value)
  tf.compat.v1.set_random_seed(seed_value)
  lstm.fit(Xtrain_arrayfe, ytrain_arrayfe, validation_data = (Xvalid_arrayfe, yvalid_arrayfe), epochs = epochs)
  y_pred = lstm.predict(Xtest)
  lstm_result_lst.append(y_pred)
  lstm.reset_states() 
end = time.time()

print("Time elapsed:", end - start)

lstm_train_time = end - start

In [ ]:
rmse_lst, mae_lst, wmape_lst, mase_lst, nrmse_lst, acc_lst, acc_lst_90 = print_originalvalues_results(lstm_result_lst, df_test_true, Xtest_fridge)

lstm_rmse = sum(rmse_lst)/len(rmse_lst)
lstm_mae = sum(mae_lst)/len(mae_lst)
lstm_wmape = sum(wmape_lst)/len(wmape_lst)
lstm_mase = sum(mase_lst)/len(mase_lst)
lstm_nrmse = sum(nrmse_lst)/len(nrmse_lst)
lstm_acc = sum(acc_lst)/len(acc_lst)
lstm_acc_90 = sum(acc_lst_90)/len(acc_lst_90)

print('Mean RMSE, MAE, WMAPE, MASE, NRMSE, ACCURACY:')
print(f'{lstm_rmse:.5f}/{lstm_mae:.5f}/{lstm_wmape:.5f}/{lstm_mase:.5f}/{lstm_nrmse:.5f}/{lstm_acc:.5f}')

lstm_lst_results = [lstm_rmse, lstm_mae, lstm_wmape, lstm_mase, lstm_nrmse, lstm_acc, lstm_acc_90, lstm_train_time]
evaluation_dict['LSTM'] = lstm_lst_results

#### BiDiLSTM

In [ ]:
start = time.time()
# Train Model 10 Fold:
bidilstm_result_lst = []
for i in [42, 10, 567, 239, 400, 1390, 380, 9, 27, 769]:

  # Define input layers
  sequential_input = Input(Xtrain_arrayfe[0].shape)
  
  # Model Architecture: LSTM
  bidilstm1 = Bidirectional(LSTM(hidd_dim))(sequential_input) #, return_sequences=True
  lr1 = LeakyReLU(alpha=lr_alpha)(bidilstm1)
  drpt1 = Dropout(drpt_rate)(lr1)

  outputs = Dense(24)(drpt1) #, activation='sigmoid'
  outputs = LeakyReLU(alpha=lr_alpha)(outputs)
  
  # Define the model
  bidilstm = Model(inputs=sequential_input, outputs=outputs)
  bidilstm.compile(loss=loss, optimizer=optimizer)


  seed_value = i
  os.environ['PYTHONHASHSEED']=str(seed_value)
  random.seed(seed_value)
  np.random.seed(seed_value)
  tf.compat.v1.set_random_seed(seed_value)
  bidilstm.fit(Xtrain_arrayfe, ytrain_arrayfe, validation_data = (Xvalid_arrayfe, yvalid_arrayfe), epochs = epochs)
  y_pred = bidilstm.predict(Xtest)
  bidilstm_result_lst.append(y_pred)
end = time.time()

print("Time elapsed:", end - start)
# save the lastm Model (save best model? + seed?)
#bidilstm.save(f'{model_path}{model_name}')
bilstm_train_time = end - start

In [ ]:
# calculate differences to original data for many models:
rmse_lst, mae_lst, wmape_lst, mase_lst, nrmse_lst, acc_lst, acc_lst_90 = print_originalvalues_results(bidilstm_result_lst, df_test_true, Xtest_fridge)

bilstm_rmse = sum(rmse_lst)/len(rmse_lst)
bilstm_mae = sum(mae_lst)/len(mae_lst)
bilstm_wmape = sum(wmape_lst)/len(wmape_lst)
bilstm_mase = sum(mase_lst)/len(mase_lst)
bilstm_nrmse = sum(nrmse_lst)/len(nrmse_lst)
bilstm_acc = sum(acc_lst)/len(acc_lst)
bilstm_acc_90 = sum(acc_lst_90)/len(acc_lst_90)

print('Mean RMSE, MAE, WMAPE, MASE, NRMSE, ACCURACY:')
print(f'{bilstm_rmse:.5f}/{bilstm_mae:.5f}/{bilstm_wmape:.5f}/{bilstm_mase:.5f}/{bilstm_nrmse:.5f}/{bilstm_acc:.5f}')

y_pred_wind_lst_bidilstm = []
for i in range(len(bidilstm_result_lst)):
  y_pred_wind = preproc_target.inverse_transform(bidilstm_result_lst[i].reshape((-1,1)))
  y_pred_wind = y_pred_wind.reshape(df_test_true_wind.shape)
  y_pred_wind_lst_bidilstm.append(y_pred_wind)
  rmse_wind, mae_wind, wmape, mase_wind, nrmse_wind, acc_wind, acc_wind_90 = evaluate_predictions_windows(df_test_true_wind, y_pred_wind, test_y = Xtest_fridge_wind, testing=False)

bilstm_lst_results = [bilstm_rmse, bilstm_mae, bilstm_wmape, bilstm_mase, bilstm_nrmse, bilstm_acc, bilstm_acc_90, bilstm_train_time]
evaluation_dict['BiLSTM'] = bilstm_lst_results

#### Seq2Seq

##### Seq2Seq from Razghandi et al. 2021

In [ ]:
# decoder output:
Xtrain_arrayfe_flip = np.flip(Xtrain_arrayfe, axis=1)[:,:,0]
Xvalid_arrayfe_flip = np.flip(Xvalid_arrayfe, axis=1)[:,:,0]
Xtest_flip = np.flip(Xtest, axis=1)[:,:,0]

optimizer1 = tf.keras.optimizers.Adam(lr=0.001)

start = time.time()
# Train Model 10 Fold:
seq2seq_result_lst = []
for i in [42, 10, 567, 239, 400, 1390, 380, 9, 27, 769]:
  # Use model in the Razghandi2021 paper Encoder Decoder
  # try with decoder return sequence and not return sequence
  encoder_input = tf.keras.Input(
      shape=Xtrain_arrayfe[0].shape, name='encoder_input') 
  
  encoder = tf.keras.layers.LSTM(hidd_dim, return_state=True, return_sequences=True)
  encoder_outputs, state_h, state_c = encoder(encoder_input)
  
  decoder_lstm = tf.keras.layers.LSTM(hidd_dim, return_sequences=True) #
  x = decoder_lstm(encoder_outputs,initial_state=[state_h, state_c])
  
  x = tf.keras.layers.LSTM(Xtrain_arrayfe.shape[1])(x)
  output = Dense(Xtrain_arrayfe.shape[1])(x)
  
  encdec = tf.keras.models.Model(inputs=encoder_input, outputs=output)
  encdec.compile(loss=loss, optimizer=optimizer1, metrics=["mse"])

  # Training Loop
  seed_value = i
  os.environ['PYTHONHASHSEED']=str(seed_value)
  random.seed(seed_value)
  np.random.seed(seed_value)
  tf.compat.v1.set_random_seed(seed_value)
  encdec.fit(Xtrain_arrayfe, y=Xtrain_arrayfe_flip, validation_data=(Xvalid_arrayfe, Xvalid_arrayfe_flip), epochs = 100)

  generator_input = tf.keras.Input(
      shape=Xtrain_arrayfe[0].shape, name='generator_input') 
  
  encoder.trainable=False
  encoder_gen, state_h, state_c = encoder(generator_input)
  
  generator_lstm = tf.keras.layers.LSTM(hidd_dim) #
  x = generator_lstm(encoder_gen,initial_state=[state_h, state_c])
  
  output = tf.keras.layers.Dense(24)(x)
  
  seq2seq = tf.keras.models.Model(inputs=generator_input, outputs=output)
  seq2seq.compile(loss=loss, optimizer=optimizer, metrics=["mse"])
  seq2seq.fit(Xtrain_arrayfe, y=ytrain_arrayfe, validation_data=(Xvalid_arrayfe, yvalid_arrayfe), epochs = epochs)

  y_pred = seq2seq.predict(Xtest)
  seq2seq_result_lst.append(y_pred)
end = time.time()

print("Time elapsed:", end - start)

s2sra_train_time = end - start

In [ ]:
# calculate differences to original data for many models:
rmse_lst, mae_lst, wmape_lst, mase_lst, nrmse_lst, acc_lst, acc_lst_90 = print_originalvalues_results(seq2seq_result_lst, df_test_true, Xtest_fridge)

s2sra_rmse = sum(rmse_lst)/len(rmse_lst)
s2sra_mae = sum(mae_lst)/len(mae_lst)
s2sra_wmape = sum(wmape_lst)/len(wmape_lst)
s2sra_mase = sum(mase_lst)/len(mase_lst)
s2sra_nrmse = sum(nrmse_lst)/len(nrmse_lst)
s2sra_acc = sum(acc_lst)/len(acc_lst)
s2sra_acc_90 = sum(acc_lst_90)/len(acc_lst_90)

print('Mean RMSE, MAE, WMAPE, MASE, NRMSE, ACCURACY:')
print(f'{s2sra_rmse:.5f}/{s2sra_mae:.5f}/{s2sra_wmape:.5f}/{s2sra_mase:.5f}/{s2sra_nrmse:.5f}/{s2sra_acc:.5f}')

#y_pred_wind_lst_seq2seq = []
#for i in range(len(seq2seq_result_lst)):
#  y_pred_wind = preproc_target.inverse_transform(seq2seq_result_lst[i].reshape((-1,1)))
#  y_pred_wind = y_pred_wind.reshape(df_test_true_wind.shape)
#  y_pred_wind_lst_seq2seq.append(y_pred_wind)
#  rmse_wind, mae_wind, wmape, mase_wind, nrmse_wind, acc_wind, acc_wind_90 = evaluate_predictions_windows(df_test_true_wind, y_pred_wind, test_y = Xtest_fridge_wind, testing=False)

s2sra_lst_results = [s2sra_rmse, s2sra_mae, s2sra_wmape, s2sra_mase, s2sra_nrmse, s2sra_acc, s2sra_acc_90, s2sra_train_time]
evaluation_dict['S2S Reversed'] = s2sra_lst_results

##### Seq2Seq with Context Features

In [ ]:
start = time.time()
# Train Model 10 Fold:
seq2seq_extra_result_lst = []
for i in [42, 10, 567, 239, 400, 1390, 380, 9, 27, 769]:

  # Use model in the Razghandi2021 paper Encoder Decoder
  # try with decoder return sequence and not return sequence
  encoder_input = tf.keras.Input(
      shape=Xtrain_arrayfe_enc[0].shape, name='encoder_input') 
  
  encoder = tf.keras.layers.LSTM(hidd_dim_context, return_state=True, return_sequences=True)
  encoder_outputs, state_h, state_c = encoder(encoder_input)
  
  decoder_inputs = tf.keras.Input(
      shape=Xtrain_arrayfe_dec[0].shape, name='decoder_inputs')
  
  decoder_lstm = tf.keras.layers.LSTM(hidd_dim_context, return_sequences=True) #
  x = decoder_lstm(decoder_inputs, initial_state=[state_h, state_c])
  
  x = tf.keras.layers.LSTM(Xtrain_arrayfe.shape[1])(x)
  output = Dense(24)(x)
  
  seq2seq_extra = tf.keras.models.Model(inputs=[encoder_input, decoder_inputs], outputs=output)
  seq2seq_extra.compile(loss=loss, optimizer=optimizer, metrics=["mse"])
  
  # Training Loop
  seed_value = i
  os.environ['PYTHONHASHSEED']=str(seed_value)
  random.seed(seed_value)
  np.random.seed(seed_value)
  tf.compat.v1.set_random_seed(seed_value)
  seq2seq_extra.fit((Xtrain_arrayfe_enc,Xtrain_arrayfe_dec), y=ytrain_arrayfe, validation_data=((Xvalid_arrayfe_enc,Xvalid_arrayfe_dec), yvalid_arrayfe), epochs = epochs)
  y_pred = seq2seq_extra.predict((Xtest_enc, Xtest_dec))
  seq2seq_extra_result_lst.append(y_pred)
end = time.time()

print("Time elapsed:", end - start)
# save the lastm Model (save best model? + seed?)
#seq2seq.save(f'{model_path}{model_name}')
s2sex_train_time = end - start

In [ ]:
# calculate differences to original data for many models:
rmse_lst, mae_lst, wmape_lst, mase_lst, nrmse_lst, acc_lst, acc_lst_90 = print_originalvalues_results(seq2seq_extra_result_lst, df_test_true, Xtest_fridge)

s2sex_rmse = sum(rmse_lst)/len(rmse_lst)
s2sex_mae = sum(mae_lst)/len(mae_lst)
s2sex_wmape = sum(wmape_lst)/len(wmape_lst)
s2sex_mase = sum(mase_lst)/len(mase_lst)
s2sex_nrmse = sum(nrmse_lst)/len(nrmse_lst)
s2sex_acc = sum(acc_lst)/len(acc_lst)
s2sex_acc_90 = sum(acc_lst_90)/len(acc_lst_90)

print('Mean RMSE, MAE, WMAPE, MASE, NRMSE, ACCURACY:')
print(f'{s2sex_rmse:.5f}/{s2sex_mae:.5f}/{s2sex_wmape:.5f}/{s2sex_mase:.5f}/{s2sex_nrmse:.5f}/{s2sex_acc:.5f}')

#y_pred_wind_lst_seq2seq_ae = []
#for i in range(len(seq2seq_extra_result_lst)):
#  y_pred_wind = preproc_target.inverse_transform(seq2seq_extra_result_lst[i].reshape((-1,1)))
#  y_pred_wind = y_pred_wind.reshape(df_test_true_wind.shape)
#  y_pred_wind_lst_seq2seq_ae.append(y_pred_wind)
#  rmse_wind, mae_wind, wmape, mase_wind, nrmse_wind, acc_wind, acc_wind_90 = evaluate_predictions_windows(df_test_true_wind, y_pred_wind, test_y = Xtest_fridge_wind, testing=False)

s2sex_lst_results = [s2sex_rmse, s2sex_mae, s2sex_wmape, s2sex_mase, s2sex_nrmse, s2sex_acc, s2sex_acc_90, s2sex_train_time]
evaluation_dict['S2S Context'] = s2sex_lst_results

#### FFNN


In [ ]:
start = time.time()
# Train Model 10 Fold:
ffnn_result_lst = []
for i in [42, 10, 567, 239, 400, 1390, 380, 9, 27, 769]:

  # Define input layers
  sequential_input = Input(X_train[0].shape)
  
  ffnn1 = Dense(hidd_dim_ffnn)(sequential_input)
  lr1 = LeakyReLU(alpha=lr_alpha)(ffnn1)
  drpt1 = Dropout(drpt_rate)(lr1)
  ffnn2 = Dense(hidd_dim_ffnn)(drpt1)
  lr2 = LeakyReLU(alpha=lr_alpha)(ffnn2)
  drpt2 = Dropout(drpt_rate)(lr2)
  ffnn3 = Dense(hidd_dim_ffnn)(drpt2)
  lr3 = LeakyReLU(alpha=lr_alpha)(ffnn3)
  drpt3 = Dropout(drpt_rate)(lr3)
  ffnn4 = Dense(hidd_dim_ffnn)(drpt3)
  lr4 = LeakyReLU(alpha=lr_alpha)(ffnn4)
  drpt4 = Dropout(drpt_rate)(lr4)
  ffnn5 = Dense(hidd_dim_ffnn)(drpt4)
  lr5 = LeakyReLU(alpha=lr_alpha)(ffnn5)
  drpt5 = Dropout(drpt_rate)(lr5)
  outputs = Dense(24)(drpt5)  
  # Define the model
  ffnn = Model(inputs=sequential_input, outputs=outputs)
  ffnn.compile(loss=loss, optimizer=optimizer)


  seed_value = i
  os.environ['PYTHONHASHSEED']=str(seed_value)
  random.seed(seed_value)
  np.random.seed(seed_value)
  tf.compat.v1.set_random_seed(seed_value)
  ffnn.fit(X_train, ytrain_arrayfe, validation_data = (X_valid, yvalid_arrayfe), epochs = epochs, batch_size=batch_size)
  y_pred = ffnn.predict(X_test)
  ffnn_result_lst.append(y_pred)

end = time.time()

print("Time elapsed:", end - start)

ffnn_train_time = end - start

In [ ]:
# calculate differences to original data for many models:
rmse_lst, mae_lst, wmape_lst, mase_lst, nrmse_lst, acc_lst, acc_lst_90 = print_originalvalues_results(ffnn_result_lst, df_test_true, Xtest_fridge)

ffnn_rmse = sum(rmse_lst)/len(rmse_lst)
ffnn_mae = sum(mae_lst)/len(mae_lst)
ffnn_wmape = sum(wmape_lst)/len(wmape_lst)
ffnn_mase = sum(mase_lst)/len(mase_lst)
ffnn_nrmse = sum(nrmse_lst)/len(nrmse_lst)
ffnn_acc = sum(acc_lst)/len(acc_lst)
ffnn_acc_90 = sum(acc_lst_90)/len(acc_lst_90)

print('Mean RMSE, MAE, WMAPE, MASE, NRMSE, ACCURACY:')
print(f'{ffnn_rmse:.5f}/{ffnn_mae:.5f}/{ffnn_wmape:.5f}/{ffnn_mase:.5f}/{ffnn_nrmse:.5f}/{ffnn_acc:.5f}')

#y_pred_wind_lst_ffnn = []
#for i in range(len(ffnn_result_lst)):
#  y_pred_wind = preproc_target.inverse_transform(ffnn_result_lst[i].reshape((-1,1)))
#  y_pred_wind = y_pred_wind.reshape(df_test_true_wind.shape)
#  y_pred_wind_lst_ffnn.append(y_pred_wind)
#  rmse_wind, mae_wind, wmape, mase_wind, nrmse_wind, acc_wind, acc_wind_90 = evaluate_predictions_windows(df_test_true_wind, y_pred_wind, test_y = Xtest_fridge_wind, testing=False)

ffnn_lst_results = [ffnn_rmse, ffnn_mae, ffnn_wmape, ffnn_mase, ffnn_nrmse, ffnn_acc, ffnn_acc_90, ffnn_train_time]
evaluation_dict['FFNN'] = ffnn_lst_results

#### CNN-LSTM

In [ ]:
start = time.time()
cnn_lstm_result_lst = []
for i in [42, 10, 567, 239, 400, 1390, 380, 9, 27, 769]:

  # Define input layers
  sequential_input = Input(Xtrain_arrayfe_cnn_lstm[0].shape)
  
  # Model Architecture: CNN-LSTM
  if Xtrain_arrayfe_cnn_lstm[0].shape[1]==1:
    conv1 = TimeDistributed(Conv1D(filters=conv_filter, kernel_size=kernel_size-1))(sequential_input)
    pl_size = 1
  else:
    conv1 = TimeDistributed(Conv1D(filters=conv_filter, kernel_size=kernel_size))(sequential_input)
  bn1 = TimeDistributed(BatchNormalization())(conv1)
  lr1 = TimeDistributed(LeakyReLU(alpha=lr_alpha))(bn1)
  drpt1 = TimeDistributed(Dropout(drpt_rate_cnn))(lr1)
  mxpl1 = TimeDistributed(MaxPooling1D(pool_size=pl_size))(drpt1)
  conv2 = TimeDistributed(Conv1D(filters=conv_filter, kernel_size=int(kernel_size-1)))(mxpl1)
  bn2 = TimeDistributed(BatchNormalization())(conv1)
  lr2 = TimeDistributed(LeakyReLU(alpha=lr_alpha))(bn2)
  drpt2 = TimeDistributed(Dropout(drpt_rate_cnn))(lr2)
  mxpl2 = TimeDistributed(MaxPooling1D(pool_size=pl_size))(drpt2)
  flat1 = TimeDistributed(Flatten())(mxpl2)
  lstm_l = LSTM(hidd_dim_cnn_lstm)(flat1)
  lrout = LeakyReLU(alpha=lr_alpha)(lstm_l)
  outputs = Dense(24)(lrout)
  
  # Define the model
  cnn_lstm = Model(inputs=sequential_input, outputs=outputs)
  cnn_lstm.compile(loss=loss, optimizer=optimizer1)


  seed_value = i
  os.environ['PYTHONHASHSEED']=str(seed_value)
  random.seed(seed_value)
  np.random.seed(seed_value)
  tf.compat.v1.set_random_seed(seed_value)
  cnn_lstm.fit(Xtrain_arrayfe_cnn_lstm, ytrain_arrayfe, validation_data = (Xvalid_arrayfe_cnn_lstm, yvalid_arrayfe), epochs = epochs)
  y_pred = cnn_lstm.predict(Xtest_cnn_lstm)
  cnn_lstm_result_lst.append(y_pred)

end = time.time()

print("Time elapsed:", end - start)

cnn_train_time = end - start

In [ ]:
# calculate differences to original data for many models:
rmse_lst, mae_lst, wmape_lst, mase_lst, nrmse_lst, acc_lst, acc_lst_90 = print_originalvalues_results(cnn_lstm_result_lst, df_test_true, Xtest_fridge)

cnn_rmse = sum(rmse_lst)/len(rmse_lst)
cnn_mae = sum(mae_lst)/len(mae_lst)
cnn_wmape = sum(wmape_lst)/len(wmape_lst)
cnn_mase = sum(mase_lst)/len(mase_lst)
cnn_nrmse = sum(nrmse_lst)/len(nrmse_lst)
cnn_acc = sum(acc_lst)/len(acc_lst)
cnn_acc_90 = sum(acc_lst_90)/len(acc_lst_90)

print('Mean RMSE, MAE, WMAPE, MASE, NRMSE, ACCURACY:')
print(f'{cnn_rmse:.5f}/{cnn_mae:.5f}/{cnn_wmape:.5f}/{cnn_mase:.5f}/{cnn_nrmse:.5f}/{cnn_acc:.5f}')

cnn_lst_results = [cnn_rmse, cnn_mae, cnn_wmape, cnn_mase, cnn_nrmse, cnn_acc, cnn_acc_90, cnn_train_time]
evaluation_dict['CNN'] = cnn_lst_results

#### Results Summary

In [ ]:
## Get all results together:
print('Scores: RMSE/MAE/MASE/nRMSE/Accuracy/Time')
for item in evaluation_dict:
  print(f'Model {item} scores: {evaluation_dict[item][0]:.2f}/{evaluation_dict[item][1]:.3f}/{evaluation_dict[item][3]:.4f}/{evaluation_dict[item][4]:.4f}/{evaluation_dict[item][5]:.4f}/{evaluation_dict[item][7]:.0f}')


Scores: RMSE/MAE/MASE/nRMSE/Accuracy/Time
Model XGBoost scores: 17.64/14.453/0.8552/0.1742/0.8010/124
Model MSVR scores: 15.72/13.517/0.7998/0.1552/0.7768/601
